## Image processing for pheno-sudoku  
### Plan  

1. Ground truth:  
    a) Bin image to 64 chunks of 256x256;  
    b) Code for rndom selection of images from the screen (x20); perform LoG on channel 3, make RGB image, R/G/B = 80/10/10 (R = ch3, membrane); Code how to normalize channels for all the images;  
    c) create masks.  
2. Training of StarDist  
3. Segmentation  
4. Clean up of single cells  
5. Clustering with unsupervised CNN  